# TUTORIAL DE SQLITE NO CONTEXTO DO SPREADMART ORÇAMENTÁRIO

## TO DO
1. Clausulas SQL para somente criar ou deletar tabelas caso não já exisitam

## Imports

In [1]:
import pandas as pd
import glob
import time
import psycopg2
import os

## Constants

In [2]:
DB_NAME = 'dadosmg'
CSV_PATH = 'datasets\\'

## Conectar ao Postgres

In [3]:
conn = psycopg2.connect("user=postgres password=postgres")
cur = conn.cursor() #cursor para realizar consultas (mais otimizado que usar con)

## Comandos básicos

In [4]:
# Execute a command: this creates a new table
cur.execute(f"CREATE TABLE {DB_NAME} (id serial PRIMARY KEY, num integer, data varchar);")

In [5]:
# Pass data to fill a query placeholders and let Psycopg perform
# the correct conversion (no more SQL injections!)
cur.execute(f"INSERT INTO {DB_NAME} (num, data) VALUES (%s, %s)", (100, "abc'def"))

In [7]:
# Query the database and obtain data as Python objects
cur.execute(f"SELECT * FROM {DB_NAME};")
row = cur.fetchone()
while row is not None:
    print(row)
    row = cur.fetchone()

(1, 100, "abc'def")


In [24]:
# SITUATIONAL: When a query produces an error you can't run another command until rolling back the transaction
conn.rollback()

In [8]:
# Make the changes to the database persistent
conn.commit()

In [9]:
# Close communication with the database
cur.close()
conn.close()

## Mostrar todas as tabelas de uma database

In [15]:
conn = psycopg2.connect("user=postgres password=postgres")
cur = conn.cursor() #cursor para realizar consultas (mais otimizado que usar con)

cur.execute("""
    SELECT table_name 
    FROM information_schema.tables 
    WHERE table_schema = 'public'""")

for table in cur.fetchall():
    print(table)

## Apagar todas as tabelas caso existam

In [16]:
cur.execute("""
    SELECT table_name 
    FROM information_schema.tables 
    WHERE table_schema = 'public'""")

res = cur.fetchall()

if res:
    for table_name in res:
        cur.execute(f"""DROP TABLE {table_name[0]} """)
        print(f"Tabela {table_name} apagada.")
else:
    print(f"Não há tabelas em {DB_NAME}")

conn.commit()

Não há tabelas em dadosmg


## Carregar arquivos CSV para tabelas com mesmo nome

In [10]:
file_paths = [i.replace('\\', '/') for i in list(glob.iglob(f'{CSV_PATH}*.csv'))]

for file in file_paths:
    _ , tail = os.path.split(file)
    table_name , file_extension = os.path.splitext(tail)
    #print(file)

    df = pd.read_csv(file, delimiter=';', decimal=',')
    df.to_sql(table_name, con, if_exists='replace', index=False) # if_exists{‘fail’, ‘replace’, ‘append’}, default ‘fail’
    print(f"Arquivo {file} carregado para tabela {table_name}")
     
con.commit()

Arquivo datasets/dm_acao.csv carregado para tabela dm_acao
Arquivo datasets/dm_categ_econ.csv carregado para tabela dm_categ_econ
Arquivo datasets/dm_elemento_desp.csv carregado para tabela dm_elemento_desp
Arquivo datasets/dm_empenho_desp_2002.csv carregado para tabela dm_empenho_desp_2002
Arquivo datasets/dm_empenho_desp_2003.csv carregado para tabela dm_empenho_desp_2003
Arquivo datasets/dm_empenho_desp_2004.csv carregado para tabela dm_empenho_desp_2004
Arquivo datasets/dm_empenho_desp_2005.csv carregado para tabela dm_empenho_desp_2005
Arquivo datasets/dm_empenho_desp_2006.csv carregado para tabela dm_empenho_desp_2006


## Mostrar todas a tabelas de uma Database

In [17]:
cur.execute("""
    SELECT table_name 
    FROM information_schema.tables 
    WHERE table_schema = 'public'""")
res = cur.fetchall()

if res:
    print(res)
else:
    print(f"Não há tabelas na database {DB_NAME}")


Não há tabelas na database dadosmg


In [30]:
tbl_agg_name = 'dm_empenho_desp'
df_agg = pd.DataFrame()
num_linhas = 0

file_paths = [i.replace('\\', '/') for i in list(glob.iglob(f'{CSV_PATH}dm_empenho_desp_*.csv'))]
file_paths

#temp_csv = con.execute(f"""SELECT * FROM '{file_paths[0]}' LIMIT 10 """)

# cria lista contendo nomes e tipos das colunas lidas em temp_csv
#table_columns = [str(temp_csv.columns[i] + ' ' + temp_csv.dtypes[i]) for i in range(len(temp_csv.columns))]

# Concatena lista de strings em uma string somente para uso na criação da tabela nova.
#table_columns = ', '.join(table_columns)


#con.execute(f"""CREATE TABLE '{tbl_agg_name}' AS SELECT * FROM df_agg """) 

for file in file_paths:
    _ , tail = os.path.split(file)
    table_name , file_extension = os.path.splitext(tail)   
    print(f'Lendo:', file)
    
    df = pd.read_csv(file, delimiter=';', decimal=',')
    df.to_sql(tbl_agg_name, con, if_exists='append', index=False) # if_exists{‘fail’, ‘replace’, ‘append’}, default ‘fail’
    
    num_linhas += len(df)
    print(f"Arquivo {file} concatenado na tabela {tbl_agg_name}\n
      


con.commit()

Lendo: datasets/dm_empenho_desp_2002.csv
Arquivo datasets/dm_empenho_desp_2002.csv concatenado na tabela dm_empenho_desp
Lendo: datasets/dm_empenho_desp_2003.csv
Arquivo datasets/dm_empenho_desp_2003.csv concatenado na tabela dm_empenho_desp
Lendo: datasets/dm_empenho_desp_2004.csv
Arquivo datasets/dm_empenho_desp_2004.csv concatenado na tabela dm_empenho_desp
Lendo: datasets/dm_empenho_desp_2005.csv
Arquivo datasets/dm_empenho_desp_2005.csv concatenado na tabela dm_empenho_desp
Lendo: datasets/dm_empenho_desp_2006.csv
Arquivo datasets/dm_empenho_desp_2006.csv concatenado na tabela dm_empenho_desp


## Fecha conexção com base de dados

In [ ]:
con.close()